# Решение системы линейных уравнений (метод Гаусса)
---
Функция `gauss_solver(A, b)` решает систему линейных уравнений
$$
A x = b,\quad A\in\mathbb{R}^{n\times n},\;b\in\mathbb{R}^{n},
$$
с помощью классического метода Гаусса с преобразованием к ступенчатому виду и обратным ходом. В случае единственного решения возвращает его, а при бесконечном множестве решений – базис пространства решений.

### Входные параметры
- $A$: матрица коэффициентов размера $n\times n$ (объект класса `Matrix`).
- $b$: столбцовый вектор правых частей размера $n\times1$ (объект класса `Matrix`).

### Выход
- `List[Matrix]`:
  - Если решение единственно, список из одного вектора-столбца $x\in\mathbb{R}^n$.
  - Если бесконечно много решений, список из:
    1. одного **частного решения** $x_{\rm part}$ при всех свободных переменных, равных нулю,
    2. набора **базисных векторов** $x^{(h)}$, соответствующих каждому свободному столбцу.

---

### Алгоритм

1. **Проверка входа**
   Убеждаемся, что $b$ имеет размер $n\times1$, иначе выбрасываем `ValueError`.

2. **Формирование расширенной матрицы**
   Строим $[A\!\mid\!b]\in\mathbb{R}^{n\times(m+1)}$:
   $$
     \text{aug}_{i,\,1..m} = A_{i,\,1..m},\quad
     \text{aug}_{i,\,m+1} = b_i.
   $$

3. **Прямой ход (приведение к ступенчатому виду)**
   - Для каждого столбца $j=1,\dots,m$ ищем первый ненулевой элемент в столбце ниже текущей строки $r$:
     $$
       \text{pivot} = \min\{i\ge r : |\text{aug}_{i,j}|>\varepsilon\}.
     $$
   - Если найден, меняем строки $r$ и `pivot`, нормируем ведущий элемент до 1:
     $$
       \text{aug}_{r,\,j..m+1} \;\gets\; \frac{\text{aug}_{r,\,j..m+1}}{\text{aug}_{r,j}}.
     $$
   - Обнуляем все элементы **ниже** в столбце $j$:
     $$
       \text{aug}_{i,\,j..m+1} \;\gets\; \text{aug}_{i,\,j..m+1}
                             - \text{aug}_{i,j}\times \text{aug}_{r,\,j..m+1},\quad i>r.
     $$
   - Добавляем $j$ в список `pivot_cols` и переходим к $r+1$.

4. **Проверка совместности**
   После прямого хода вычисляем ранг $\rho = |\text{pivot\_cols}|$.
   Если найдётся строка вида
   $$
     [\,0,\dots,0\mid c\,],\;c\neq0,
   $$
   система **несовместна** (выбрасываем `ValueError`).

5. **Обратный ход и построение решений**
   - Если $\rho = m$, система имеет **единственное решение**:
     \[
       x_j = \bigl(\text{строка }r\text{ после вычитаний}\bigr)_j,\quad j\in\text{pivot\_cols},
     \]
     возвращаем новый `Matrix`-столбец $x$.
   - Иначе $\rho < m$ — **бесконечно много решений**.
     1. Находим **частное решение** $x_{\rm part}$, полагая все свободные переменные $=0$.
     2. Для каждого свободного столбца $f$ строим **базисный вектор** $x^{(f)}$, полагая $x_f=1$, остальные свободные $=0$, и вычисляя зависимые через обратный ход:
        $$
          x^{(f)}_j = -\sum_{k>j} \text{aug}_{r,j,k}\,x^{(f)}_k,\quad j\in\text{pivot\_cols}.
        $$
     3. Собираем список `[x_part, x^{(f_1)}, x^{(f_2)}, …]`.

---

In [1]:
from typing import List
from src.Matrix import Matrix

In [2]:
def gauss_solver(A: 'Matrix', b: 'Matrix') -> List['Matrix']:
    """
    Вход:
    A: матрица коэффициентов (n×n). Используется класс Matrix.
    b: вектор правых частей (n×1)
    Выход:
    list[Matrix]: список базисных векторов решения системы
    Raises:
        ValueError: если система несовместна
    """
    n = A.rows
    m = A.cols
    if b.rows != n or b.cols != 1:
        raise ValueError("b должен быть столбцовым вектором длины n")

    # Формируем расширенную матрицу [A|b]
    aug = [A.data[i][:] + [b.data[i][0]] for i in range(n)]
    EPS = 1e-9  # порог для нуля
    pivot_cols = []  # список индексов ведущих столбцов
    row = 0

    # Прямой ход
    for col in range(m):
        # Поиск ненулевого ведущего элемента в текущем столбце
        pivot = row
        while pivot < n and abs(aug[pivot][col]) < EPS:
            pivot += 1
        if pivot == n:
            # В этом столбце ведущего элемента нет
            continue
        # Меняем местами текущую строку и строку с найденным ведущим элементом
        if pivot != row:
            aug[row], aug[pivot] = aug[pivot], aug[row]
        pivot_cols.append(col)
        pivot_val = aug[row][col]
        # Нормируем ведущий элемент к 1 (не обязательно, но удобно)
        if abs(pivot_val - 1.0) > 1e-12:
            for j in range(col, m + 1):
                aug[row][j] /= pivot_val
        # Обнуление элементов ниже ведущего в текущем столбце
        for i in range(row + 1, n):
            factor = aug[i][col]
            if abs(factor) > EPS:
                for j in range(col, m + 1):
                    aug[i][j] -= factor * aug[row][j]
        row += 1
        if row == n:
            break
    # Теперь матрица приведена к ступенчатому виду
    rank = len(pivot_cols)
    # Проверяем на несовместность: если есть строка [0...0 | c], c != 0
    for i in range(rank, n):
        # все коэффициенты 0?
        if all(abs(aug[i][j]) < EPS for j in range(m)) and abs(aug[i][m]) > EPS:
            raise ValueError("Система несовместна")
    solutions: List[Matrix] = []

    # Обратный ход
    x = [0.0] * m
    if rank == m:
        # Единственное решение
        for r in range(m - 1, -1, -1):
            j = pivot_cols[r]  # индекс столбца с ведущим элементом на этой строке
            s = aug[r][m]
            for k in range(j + 1, m):
                s -= aug[r][k] * x[k]
            x[j] = s / (aug[r][j] if abs(aug[r][j]) > EPS else 1.0)
        sol_vec = Matrix([[x[j]] for j in range(m)])  # столбец-матрица решения
        solutions.append(sol_vec)
    else:
        # Бесконечно много решений: параметризуем свободные переменные
        free_cols = [j for j in range(m) if j not in pivot_cols]
        # Находим частное решение (при всех свободных = 0)
        x_part = [0.0] * m
        for r in range(rank - 1, -1, -1):
            j = pivot_cols[r]
            s = aug[r][m]
            for k in range(j + 1, m):
                s -= aug[r][k] * (x_part[k] if k not in free_cols else 0.0)
            x_part[j] = s / (aug[r][j] if abs(aug[r][j]) > EPS else 1.0)
        solutions.append(Matrix([[x_part[j]] for j in range(m)]))

        # Формируем базисные решения, положив по очереди одну свободную переменную = 1, остальные = 0
        for f in free_cols:
            x_h = [0.0] * m
            x_h[f] = 1.0
            for r in range(rank - 1, -1, -1):
                j = pivot_cols[r]
                s = 0.0
                for k in range(j + 1, m):
                    s -= aug[r][k] * (x_h[k])
                x_h[j] = s / (aug[r][j] if abs(aug[r][j]) > EPS else 1.0)
            solutions.append(Matrix([[x_h[j]] for j in range(m)]))
    return solutions

---